#Statistical approach

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import fetch_20newsgroups

# Load the 20 Newsgroups dataset
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert dataset into a DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
label_names = newsgroups.target_names  # Category names

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Choose classifier (MultinomialNB or LogisticRegression)
classifier = MultinomialNB()
# classifier = LogisticRegression(max_iter=2000)

# Train classifier
classifier.fit(X_train_tfidf, y_train)

# Predict on test set
y_pred = classifier.predict(X_test_tfidf)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_test, y_pred, target_names=label_names))

Accuracy: 0.88
Classification Report:
                        precision    recall  f1-score   support

          alt.atheism       0.89      0.79      0.84       172
        comp.graphics       0.89      0.91      0.90       216
   rec.sport.baseball       0.83      0.97      0.90       178
            sci.space       0.91      0.81      0.86       185
talk.politics.mideast       0.87      0.89      0.88       188

             accuracy                           0.88       939
            macro avg       0.88      0.87      0.87       939
         weighted avg       0.88      0.88      0.88       939



# Neural Network Approach

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import fetch_20newsgroups

# Load the 20 Newsgroups dataset
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert dataset into a DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
label_names = newsgroups.target_names  # Category names

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Define neural network classifier
classifier = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500, random_state=42)

# Train classifier
classifier.fit(X_train_tfidf, y_train)

# Predict on test set
y_pred = classifier.predict(X_test_tfidf)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_test, y_pred, target_names=label_names))

Accuracy: 0.87
Classification Report:
                        precision    recall  f1-score   support

          alt.atheism       0.90      0.78      0.84       172
        comp.graphics       0.90      0.90      0.90       216
   rec.sport.baseball       0.84      0.94      0.89       178
            sci.space       0.89      0.83      0.86       185
talk.politics.mideast       0.84      0.89      0.87       188

             accuracy                           0.87       939
            macro avg       0.87      0.87      0.87       939
         weighted avg       0.87      0.87      0.87       939



In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from itertools import chain

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the 20 Newsgroups dataset
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert dataset into a DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
label_names = newsgroups.target_names  # Category names

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Tokenize text
def tokenize(text):
    return text.lower().split()

# Build vocabulary manually
counter = Counter(chain.from_iterable(map(tokenize, X_train)))
vocab = {word: i + 2 for i, (word, _) in enumerate(counter.most_common())}  # Start indexing from 2
vocab["<PAD>"] = 0
vocab["<OOV>"] = 1

# Convert text to tensor
def text_pipeline(text):
    return torch.tensor([vocab.get(token, vocab["<OOV>"]) for token in tokenize(text)], dtype=torch.long)

# Create Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = torch.tensor(labels.values, dtype=torch.long)  # Ensure labels are tensors

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Preprocess datasets
X_train_tensor = [text_pipeline(text) for text in X_train]
X_test_tensor = [text_pipeline(text) for text in X_test]

# Pad sequences
X_train_padded = pad_sequence(X_train_tensor, batch_first=True, padding_value=vocab["<PAD>"])
X_test_padded = pad_sequence(X_test_tensor, batch_first=True, padding_value=vocab["<PAD>"])

# Create DataLoader
train_dataset = TextDataset(X_train_padded, y_train)
test_dataset = TextDataset(X_test_padded, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define RNN model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=vocab["<PAD>"])
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

# Initialize model
vocab_size = len(vocab)
embed_size = 128
hidden_size = 128
output_size = len(categories)

model = RNNModel(vocab_size, embed_size, hidden_size, output_size).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
epochs = 5
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct, total = 0, 0
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_accuracy = correct / total
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Evaluate model
model.eval()
y_pred, y_true = [], []
with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)  # Move data to GPU
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

accuracy = sum(np.array(y_pred) == np.array(y_true)) / len(y_true)
print(f'Final Test Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_true, y_pred, target_names=label_names))

Using device: cuda
Epoch 1, Loss: 189.8386, Accuracy: 0.2067
Epoch 2, Loss: 189.5745, Accuracy: 0.2064
Epoch 3, Loss: 189.5594, Accuracy: 0.2099
Epoch 4, Loss: 189.4750, Accuracy: 0.2176
Epoch 5, Loss: 189.5154, Accuracy: 0.2086
Final Test Accuracy: 0.19
Classification Report:
                        precision    recall  f1-score   support

          alt.atheism       0.00      0.00      0.00       172
        comp.graphics       0.00      0.00      0.00       216
   rec.sport.baseball       0.19      1.00      0.32       178
            sci.space       0.00      0.00      0.00       185
talk.politics.mideast       0.00      0.00      0.00       188

             accuracy                           0.19       939
            macro avg       0.04      0.20      0.06       939
         weighted avg       0.04      0.19      0.06       939



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from itertools import chain

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the 20 Newsgroups dataset
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert dataset into a DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
label_names = newsgroups.target_names  # Category names

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Tokenize text
def tokenize(text):
    return text.lower().split()

# Build vocabulary manually
counter = Counter(chain.from_iterable(map(tokenize, X_train)))
vocab = {word: i + 2 for i, (word, _) in enumerate(counter.most_common())}  # Start indexing from 2
vocab["<PAD>"] = 0
vocab["<OOV>"] = 1

# Convert text to tensor
def text_pipeline(text):
    return torch.tensor([vocab.get(token, vocab["<OOV>"]) for token in tokenize(text)], dtype=torch.long)

# Create Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = torch.tensor(labels.values, dtype=torch.long)  # Ensure labels are tensors

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Preprocess datasets
X_train_tensor = [text_pipeline(text) for text in X_train]
X_test_tensor = [text_pipeline(text) for text in X_test]

# Pad sequences
X_train_padded = pad_sequence(X_train_tensor, batch_first=True, padding_value=vocab["<PAD>"])
X_test_padded = pad_sequence(X_test_tensor, batch_first=True, padding_value=vocab["<PAD>"])

# Create DataLoader
train_dataset = TextDataset(X_train_padded, y_train)
test_dataset = TextDataset(X_test_padded, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=vocab["<PAD>"])
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        return self.fc(hidden[-1])

# Initialize model
vocab_size = len(vocab)
embed_size = 128
hidden_size = 128
output_size = len(categories)

model = LSTMModel(vocab_size, embed_size, hidden_size, output_size).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
epochs = 5
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct, total = 0, 0
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_accuracy = correct / total
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Evaluate model
model.eval()
y_pred, y_true = [], []
with torch.no_grad():
    for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)  # Move data to GPU
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

accuracy = sum(np.array(y_pred) == np.array(y_true)) / len(y_true)
print(f'Final Test Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_true, y_pred, target_names=label_names))

Using device: cuda
Epoch 1, Loss: 189.6290, Accuracy: 0.2134
Epoch 2, Loss: 189.5242, Accuracy: 0.2147
Epoch 3, Loss: 189.5330, Accuracy: 0.2054
Epoch 4, Loss: 189.4989, Accuracy: 0.2144
Epoch 5, Loss: 189.4678, Accuracy: 0.2123
Epoch 6, Loss: 189.4669, Accuracy: 0.2142
Epoch 7, Loss: 189.4432, Accuracy: 0.2094
Epoch 8, Loss: 189.4561, Accuracy: 0.2123
Epoch 9, Loss: 189.4561, Accuracy: 0.2139
Epoch 10, Loss: 189.4346, Accuracy: 0.2126
Epoch 11, Loss: 189.4329, Accuracy: 0.2112
Epoch 12, Loss: 189.4836, Accuracy: 0.2176
Epoch 13, Loss: 189.4752, Accuracy: 0.2040
Epoch 14, Loss: 189.4369, Accuracy: 0.2118
Epoch 15, Loss: 189.4335, Accuracy: 0.2088
Final Test Accuracy: 0.19
Classification Report:
                        precision    recall  f1-score   support

          alt.atheism       0.00      0.00      0.00       172
        comp.graphics       0.00      0.00      0.00       216
   rec.sport.baseball       0.19      1.00      0.32       178
            sci.space       0.00      0.00

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Pre-trained Model

## BERT only

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the 20 Newsgroups dataset
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert dataset into a DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
label_names = newsgroups.target_names  # Category names

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text and convert to tensor
def text_pipeline(text):
    encoded = tokenizer(text, padding='max_length', truncation=True, max_length=256, return_tensors='pt')
    return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

# Create Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = [text_pipeline(text) for text in texts]
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx][0], self.texts[idx][1], self.labels[idx]

# Create DataLoader
test_dataset = TextDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=16)

# Define BERT model
class BERT_Model(nn.Module):
    def __init__(self, output_size):
        super(BERT_Model, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, output_size)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT parameters
            bert_output = self.bert(input_ids, attention_mask=attention_mask).pooler_output
        return self.fc(bert_output)

# Initialize model
output_size = len(categories)

model = BERT_Model(output_size).to(device)

# Evaluate model
model.eval()
y_pred, y_true = [], []
with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

accuracy = sum(np.array(y_pred) == np.array(y_true)) / len(y_true)
print(f'Final Test Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_true, y_pred, target_names=label_names))

Using device: cuda
Final Test Accuracy: 0.19
Classification Report:
                        precision    recall  f1-score   support

          alt.atheism       0.18      0.95      0.31       172
        comp.graphics       0.00      0.00      0.00       216
   rec.sport.baseball       0.32      0.07      0.11       178
            sci.space       0.00      0.00      0.00       185
talk.politics.mideast       1.00      0.01      0.01       188

             accuracy                           0.19       939
            macro avg       0.30      0.21      0.09       939
         weighted avg       0.29      0.19      0.08       939



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## BERT + LSTM

In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from itertools import chain

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the 20 Newsgroups dataset
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert dataset into a DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
label_names = newsgroups.target_names  # Category names

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text and convert to tensor
def text_pipeline(text):
    encoded = tokenizer(text, padding='max_length', truncation=True, max_length=256, return_tensors='pt')
    return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

# Create Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = [text_pipeline(text) for text in texts]
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx][0], self.texts[idx][1], self.labels[idx]

# Create DataLoader
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Define BERT + LSTM model
class BERT_LSTM_Model(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(BERT_LSTM_Model, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.lstm = nn.LSTM(768, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT parameters
            bert_output = self.bert(input_ids, attention_mask=attention_mask).last_hidden_state
        _, (hidden, _) = self.lstm(bert_output)
        return self.fc(hidden[-1])

# Initialize model
hidden_size = 128
output_size = len(categories)

model = BERT_LSTM_Model(hidden_size, output_size).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
epochs = 5
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct, total = 0, 0
    for input_ids, attention_mask, labels in train_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_accuracy = correct / total
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Evaluate model
model.eval()
y_pred, y_true = [], []
with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

accuracy = sum(np.array(y_pred) == np.array(y_true)) / len(y_true)
print(f'Final Test Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_true, y_pred, target_names=label_names))

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1, Loss: 174.4217, Accuracy: 0.7299
Epoch 2, Loss: 123.2033, Accuracy: 0.8210
Epoch 3, Loss: 106.8013, Accuracy: 0.8370
Epoch 4, Loss: 93.0590, Accuracy: 0.8532
Epoch 5, Loss: 80.2029, Accuracy: 0.8791
Final Test Accuracy: 0.85
Classification Report:
                        precision    recall  f1-score   support

          alt.atheism       0.82      0.72      0.77       172
        comp.graphics       0.95      0.86      0.90       216
   rec.sport.baseball       0.80      0.96      0.87       178
            sci.space       0.86      0.82      0.84       185
talk.politics.mideast       0.84      0.91      0.87       188

             accuracy                           0.85       939
            macro avg       0.85      0.85      0.85       939
         weighted avg       0.86      0.85      0.85       939



## BERT fine-tuned

In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the 20 Newsgroups dataset
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 'sci.space', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Convert dataset into a DataFrame
df = pd.DataFrame({'text': newsgroups.data, 'label': newsgroups.target})
label_names = newsgroups.target_names  # Category names

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize text and convert to tensor
def text_pipeline(text):
    encoded = tokenizer(text, padding='max_length', truncation=True, max_length=256, return_tensors='pt')
    return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

# Create Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = [text_pipeline(text) for text in texts]
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx][0], self.texts[idx][1], self.labels[idx]

# Create DataLoader
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Define BERT model
class BERT_Model(nn.Module):
    def __init__(self, output_size):
        super(BERT_Model, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.fc = nn.Linear(768, output_size)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT parameters
            bert_output = self.bert(input_ids, attention_mask=attention_mask).pooler_output
        return self.fc(bert_output)

# Initialize model
output_size = len(categories)

model = BERT_Model(output_size).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
epochs = 5
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    correct, total = 0, 0
    for input_ids, attention_mask, labels in train_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_accuracy = correct / total
    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

# Evaluate model
model.eval()
y_pred, y_true = [], []
with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

accuracy = sum(np.array(y_pred) == np.array(y_true)) / len(y_true)
print(f'Final Test Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_true, y_pred, target_names=label_names))

Using device: cuda
Epoch 1, Loss: 339.3647, Accuracy: 0.4009
Epoch 2, Loss: 278.8822, Accuracy: 0.5879
Epoch 3, Loss: 244.5542, Accuracy: 0.6668
Epoch 4, Loss: 228.2336, Accuracy: 0.6801
Epoch 5, Loss: 209.1781, Accuracy: 0.7152
Final Test Accuracy: 0.72
Classification Report:
                        precision    recall  f1-score   support

          alt.atheism       0.88      0.25      0.39       172
        comp.graphics       0.84      0.83      0.84       216
   rec.sport.baseball       0.85      0.88      0.87       178
            sci.space       0.66      0.71      0.68       185
talk.politics.mideast       0.56      0.89      0.69       188

             accuracy                           0.72       939
            macro avg       0.76      0.71      0.69       939
         weighted avg       0.76      0.72      0.70       939

